# Getting the list of compounds used in JUMP CP effort

In [1]:
import pandas as pd

InChI keys of compounds from jump-cell painting dataset (https://github.com/jump-cellpainting/datasets)

In [2]:
jump_cp_inchlkey = pd.read_csv('./compound.csv')
jump_cp_inchlkey.shape

(116753, 3)

In [3]:
jump_cp_inchlkey.head()

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...


Converting InChI keys to PubChem Compound IDs (https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi)

In [3]:
cids = pd.read_csv('./jump_cell_painting_cids.csv', index_col = 0)

In [4]:
cids.shape

(107332, 1)

In [5]:
cids.head()

,cid
0,110094592
1,5076487
2,1697
3,52934829
4,72019641


# Collecting compound descriptions

In [2]:
import json
import os
import requests
import urllib
from urllib.request import urlopen
from tqdm import tqdm_notebook
from tqdm import tqdm
#from fake_useragent import UserAgent
#from bs4 import BeautifulSoup
import time
import shutup; shutup.please()

In [17]:
data = []
for i in tqdm_notebook(range(cids.shape[0])):
    try:
        CID = str(cids.iloc[i, 0])
        page_link = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + CID + '/description/JSON'
        json_url = urlopen(page_link)
        json_data = json.loads(json_url.read())
        if 'InformationList' in json_data and 'Information' in json_data['InformationList']:
            for info in json_data['InformationList']['Information']:
                data.append({
                    'CID': CID,
                    'Description': info.get('Description', None)
                })
        #time.sleep(1)
    except:
        continue

  0%|          | 0/107332 [00:00<?, ?it/s]

In [18]:
descriptions = pd.DataFrame(data).dropna(subset = 'Description').reset_index(drop=True)
descriptions.shape

(4094, 2)

In [19]:
descriptions.head()

,CID,Description
0,1697,"4,5-dianilinophthalimide is phthalimide substi..."
1,755673,"7-hydroxy-2,3,4,5-tetrahydrobenzofuro[2,3-c]az..."
2,15160711,Ovalitenin B is a butanone.
3,176870,Erlotinib is a quinazoline compound having a (...
4,656344,"2-[[5-(4-methylphenyl)-1,3,4-oxadiazol-2-yl]th..."


In [11]:
descriptions.to_csv('./cmpd_descriptions.csv')

# Collecting pharmacology data

In [34]:
data = []
for i in tqdm_notebook(range(cids.shape[0])):
    try:
        CID = str(cids.iloc[i, 0])
        page_link = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/' + CID + '/JSON/'
        request = urllib.request.Request(page_link)
        response = urllib.request.urlopen(request)
        json_data = json.load(response, strict = False)
        cmpd_section = json_data['Record']['Section']
        for i in cmpd_section:
            if i['TOCHeading'].startswith('Pharmacology and Biochemistry'):
                pharm_data = i['Section']
                string = json.dumps(pharm_data)
                data.append({
                    'CID': CID,
                    'Pharmacology information': string
                })
    except:
        continue

  0%|          | 0/107332 [00:00<?, ?it/s]

In [35]:
pharmacology = pd.DataFrame(data)
pharmacology.head()

,CID,Pharmacology information
0,1697,"[{""TOCHeading"": ""MeSH Pharmacological Classifi..."
1,31789,"[{""TOCHeading"": ""MeSH Pharmacological Classifi..."
2,176870,"[{""TOCHeading"": ""MeSH Pharmacological Classifi..."
3,226036,"[{""TOCHeading"": ""Human Metabolite Information""..."
4,2431,"[{""TOCHeading"": ""Pharmacodynamics"", ""Descripti..."


In [36]:
pharmacology.shape

(2868, 2)

In [37]:
pharmacology.to_csv('./cmpd_pharmacology_info.csv')

# Collecting bioactivity data

In [43]:
data = []
for i in tqdm_notebook(range(80000, 107332)):
  try:
    CID = str(cids.iloc[i, 0])
    url = 'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22order%22:[%22acvalue,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22pubchem_cid_' + CID + '_bioactivity%22,%22nullatbottom%22:1,%22where%22:{%22ands%22:[{%22cid%22:%22' + CID + '%22}]}}'
    json_url = urlopen(url)
    json_data = json.loads(json_url.read(), strict = False)
    for assay in json_data:
      if assay.get('activityid') == 'Inactive':
        pass
      else:
        data.append ({
        'CID' : CID,
        'activity' : assay.get('activityid'),
        'text' : assay.get('aidname')
      })
  except:
    continue

  0%|          | 0/27332 [00:00<?, ?it/s]

In [ ]:
assays = pd.DataFrame(data)

In [45]:
assays.head()

,CID,activity,text
0,2826596,Active,Dicer-mediated maturation of pre-microRNA
1,4603394,Unspecified,Antiplasmodial activity against chloroquine-re...
2,4891910,Unspecified,XC50 for P. falciparum 3D7
3,4891910,Active,Inhibition of Rac1-dependent cell adhesion in ...
4,4891910,Unspecified,Inhibition of PDGF-BB-induced Rac activation i...


In [61]:
assays.shape

(1948536, 3)

In [ ]:
assays.to_csv('./bioassays.csv')